In [2]:
#Install these packages
"""
pip3.11 install unsloth
pip3.11 install transformers==4.56.2
pip3.11 install --no-deps trl==0.22.2
pip3.11 install sentencepiece protobuf datasets==4.3.0 "huggingface_hub>=0.34.0" hf_transfer
pip3.11 install accelerate
pip3.11 install peft
pip3.11 install bitsandbytes
"""


'\npip3.11 install unsloth\npip3.11 install transformers==4.56.2\npip3.11 install --no-deps trl==0.22.2\npip3.11 install sentencepiece protobuf datasets==4.3.0 "huggingface_hub>=0.34.0" hf_transfer\npip3.11 install accelerate\npip3.11 install peft\npip3.11 install bitsandbytes\n'

In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 2048,   # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False, 
    # token = "hf_...",      
)

In [ ]:
#LoRA Adapters to only update few parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,  
    lora_dropout = 0, 
    bias = "none",    
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  
    loftq_config = None,  # And LoftQ
)